<a href="https://colab.research.google.com/github/Elvin26/DATATHON_4/blob/main/%EA%B6%8C%EC%A7%80%EC%98%88/%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%86%A4_4%ED%8C%80_shinhan_%EC%A0%84%EC%B2%98%EB%A6%AC_%EC%99%84%EB%A3%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#라이브러리 및 데이터 불러오기

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib.ticker import FuncFormatter
import matplotlib as mpl
import seaborn as sns
import matplotlib.patches as patches

import requests
from tqdm import tqdm

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [7]:
df1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/shinhan/big_data_set1_f_ko.csv")  #기본 데이터셋 1
df_2 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/shinhan/big_data_set2_f_ko.csv") #기본 데이터셋 2
df_3 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/shinhan/big_data_set3_f_ko.csv") #기본 데이터셋 3

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/shinhan/merged_filled_zone.csv", encoding="cp949") #상권 클러스터 데이터셋

df_m = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/shinhan/가맹점_고객비중_24개월.csv") #머신 러닝 데이터셋

# 지역 전처리

In [8]:
# 1️⃣ 카카오 REST API 키
KAKAO_REST_API_KEY = '####'  # 본인 키 그대로 사용

# 2️⃣ CSV 불러오기 (위경도 누락된 데이터)
df_temp = pd.read_csv("temp2.csv")

# 3️⃣ 주소 → 위도·경도 변환 함수
def convert_address_to_coordinates(address):
    """
    입력받은 주소를 WGS84 좌표계 위도(lat), 경도(lng)로 변환 (카카오 로컬 API)
    """
    try:
        url = 'https://dapi.kakao.com/v2/local/search/address.json'
        headers = {'Authorization': 'KakaoAK ' + KAKAO_REST_API_KEY}
        params = {'query': address}

        r = requests.get(url, headers=headers, params=params)
        if r.status_code == 200:
            docs = r.json().get("documents", [])
            if docs:
                lng = float(docs[0]["address"]["x"])
                lat = float(docs[0]["address"]["y"])
                return pd.Series([lat, lng])
            else:
                return pd.Series([None, None])
        else:
            print(f"HTTP 오류 ({r.status_code}) - {address}")
            return pd.Series([None, None])

    except Exception as e:
        print(f"⚠️ 오류 발생 ({address}): {e}")
        return pd.Series([None, None])

tqdm.pandas(desc="카카오맵 주소 변환 중...")

# 5️⃣ 위도·경도 변환 실행
df_temp[['위도', '경도']] = df_temp['가맹점주소'].progress_apply(convert_address_to_coordinates)

# 6️⃣ 변환 결과 확인
print(df_temp.head())

# 7️⃣ CSV로 저장
df_temp.to_csv("temp2_with_coords.csv", index=False, encoding='utf-8-sig')
print("✅ 위도·경도 변환 완료 및 temp2_with_coords.csv 저장 완료!")


FileNotFoundError: [Errno 2] No such file or directory: 'temp2.csv'

In [ ]:
import pandas as pd

key = '가맹점구분번호'
for df in (df1, df_temp):
    df[key] = df[key].astype(str).str.strip()

coords = df_temp[[key, '위도', '경도']].dropna(how='all')

# 병합 (df1 기준 왼쪽조인)
merged = df1.merge(coords, on=key, how='left', suffixes=('', '_new'))

# 기존 df1의 결측만 df_temp 값으로 채우기
merged['위도'] = merged['위도'].combine_first(merged['위도_new'])
merged['경도'] = merged['경도'].combine_first(merged['경도_new'])

merged = merged.drop(columns=['위도_new', '경도_new'])

# 6️⃣ 결과 확인 및 저장
print("✅ 업데이트된 df1 미리보기:")
print(merged[['가맹점구분번호', '가맹점주소', '위도', '경도']].head())

merged.to_csv("df1_updated.csv", index=False, encoding="utf-8-sig")
print("✅ df1_updated.csv 저장 완료!")


# 공통 전처리

In [9]:
df_1 = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/shinhan/df1_updated.csv") #업데이트 된 데이터셋 1 다시 불러오기

In [10]:
df_1.head()

,가맹점구분번호,가맹점주소,가맹점명,브랜드구분코드,MCT_SIGUNGU_NM,업종,상권,개설일,폐업일,위도,경도
0,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,20130320,NaN,37.569723,127.039314
1,4D039EA8B7,서울 성동구 마장로37길 53,대보**,NaN,서울 성동구,축산물,마장동,20131122,NaN,37.570200,127.041487
2,0074C4990A,서울 성동구 마장로35다길 9,대용**,NaN,서울 성동구,축산물,마장동,20140512,NaN,37.569729,127.040521
3,68308F2746,서울 성동구 마장로35다길 18,통일**,NaN,서울 성동구,축산물,마장동,20151124,NaN,37.569051,127.040658
4,4117EDDE9C,서울 성동구 마장로31길 28-21,한울**,NaN,서울 성동구,축산물,마장동,20151211,NaN,37.568866,127.038536


In [11]:
df_1.shape

(4172, 11)

In [12]:
df_2.head()

,가맹점구분번호,기준년월,운영개월수_구간,월매출_구간,월매출건수_구간,월고객수_구간,객단가_구간,배달매출비율,배달매출비율.1,12개월_업종대비_평균매출_비율,12개월_업종대비_평균고객수_비율,12개월_업종내_매출_백분위,12개월_상권내_매출_백분위,12개월_상권내_가맹점_비중,12개월_동일상권_폐업가맹점_비중
0,000F03E44A,202404,4_50-75%,5_75-90%,5_75-90%,5_75-90%,5_75-90%,1_상위1구간,-999999.9,2.6,10.6,93.8,71.5,16.7,7.8
1,000F03E44A,202312,4_50-75%,6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),6_90%초과(하위 10% 이하),NaN,-999999.9,0.0,0.0,94.8,73.4,16.6,7.2
2,002816BA73,202404,2_10-25%,3_25-50%,4_50-75%,4_50-75%,2_10-25%,6_상위6구간(하위1구간),-999999.9,96.6,40.8,15.6,20.0,17.5,5.2
3,002816BA73,202411,2_10-25%,3_25-50%,4_50-75%,4_50-75%,2_10-25%,6_상위6구간(하위1구간),-999999.9,108.0,46.4,16.8,19.8,16.9,6.9
4,002816BA73,202406,2_10-25%,4_50-75%,4_50-75%,4_50-75%,2_10-25%,1_상위1구간,-999999.9,74.2,38.1,16.0,19.9,17.3,5.6


In [13]:
df_3.head()

,가맹점구분번호,기준년월,남성20대이하_비중,남성30대_비중,남성40대_비중,남성50대_비중,남성60대이상_비중,여성20대이하_비중,여성30대_비중,여성40대_비중,여성50대_비중,여성60대이상_비중,재방문고객_비중,신규고객_비중,거주지고객_비중,직장이고객_비중,유동인구고객_비중
0,0305234DDB,202311,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.00,0.0,-999999.9,-999999.9,-999999.9
1,0495B069FF,202403,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.00,25.0,0.0,0.0,100.0
2,0495B069FF,202405,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.33,0.0,-999999.9,-999999.9,-999999.9
3,0495B069FF,202406,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.33,0.0,-999999.9,-999999.9,-999999.9
4,055EDDDD01,202410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,33.33,0.0,-999999.9,-999999.9,-999999.9


In [14]:
df_merged = pd.merge(df_2, df_3, on=['가맹점구분번호','기준년월'], how='left') #2번과, 3번 병합
df_merged = pd.merge(df_1, df_merged, on='가맹점구분번호',how='left') #위 병합과, 1번 병합

In [15]:
df_merged.shape

(86278, 40)

In [16]:
df_merged.head()

,가맹점구분번호,가맹점주소,가맹점명,브랜드구분코드,MCT_SIGUNGU_NM,업종,상권,개설일,폐업일,위도,경도,기준년월,운영개월수_구간,월매출_구간,월매출건수_구간,월고객수_구간,객단가_구간,배달매출비율,배달매출비율.1,12개월_업종대비_평균매출_비율,12개월_업종대비_평균고객수_비율,12개월_업종내_매출_백분위,12개월_상권내_매출_백분위,12개월_상권내_가맹점_비중,12개월_동일상권_폐업가맹점_비중,남성20대이하_비중,남성30대_비중,남성40대_비중,남성50대_비중,남성60대이상_비중,여성20대이하_비중,여성30대_비중,여성40대_비중,여성50대_비중,여성60대이상_비중,재방문고객_비중,신규고객_비중,거주지고객_비중,직장이고객_비중,유동인구고객_비중
0,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,20130320,NaN,37.569723,127.039314,202405,2_10-25%,3_25-50%,4_50-75%,4_50-75%,2_10-25%,6_상위6구간(하위1구간),-999999.9,70.4,33.9,27.2,20.8,12.3,5.6,4.1480,9.044,11.0840,19.4480,24.2760,1.9520,4.256,5.2160,9.1520,11.4240,18.82,6.47,50.0,7.1,42.9
1,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,20130320,NaN,37.569723,127.039314,202304,2_10-25%,4_50-75%,4_50-75%,4_50-75%,2_10-25%,6_상위6구간(하위1구간),-999999.9,74.6,22.9,11.5,21.3,11.3,6.0,3.9864,11.174,9.4224,16.9120,18.9052,2.6136,7.326,6.1776,11.0880,12.3948,16.51,5.05,25.0,6.3,68.8
2,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,20130320,NaN,37.569723,127.039314,202308,2_10-25%,3_25-50%,4_50-75%,4_50-75%,2_10-25%,1_상위1구간,-999999.9,85.6,22.1,11.5,20.6,12.0,5.4,3.9060,11.214,10.6470,17.4510,19.7820,2.2940,6.586,6.2530,10.2490,11.6180,16.98,6.60,17.6,0.0,82.4
3,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,20130320,NaN,37.569723,127.039314,202402,2_10-25%,3_25-50%,4_50-75%,4_50-75%,1_10%이하,1_상위1구간,-999999.9,90.5,35.8,27.8,22.2,12.3,5.0,4.7056,10.380,10.3800,21.8672,21.8672,2.0944,4.620,4.6200,9.7328,9.7328,16.95,9.60,15.8,5.3,78.9
4,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,20130320,NaN,37.569723,127.039314,202406,2_10-25%,3_25-50%,4_50-75%,4_50-75%,1_10%이하,1_상위1구간,-999999.9,81.1,27.5,26.9,21.0,12.2,5.6,3.9345,7.869,10.8360,18.1245,23.6715,2.1655,4.331,5.9640,9.9755,13.0285,20.96,4.79,26.7,0.0,73.3


In [17]:
df_merged['개설일'] = pd.to_datetime(df_merged['개설일'], format='%Y%m%d')
df_merged['폐업일'] = pd.to_datetime(df_merged['폐업일'], format='%Y%m%d', errors='coerce') # 오류 발생 시 NaT으로 처리
df_merged['기준년월_dt'] = pd.to_datetime(df_merged['기준년월'], format='%Y%m').dt.to_period('M')

In [18]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86278 entries, 0 to 86277
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   가맹점구분번호             86278 non-null  object        
 1   가맹점주소               86278 non-null  object        
 2   가맹점명                86278 non-null  object        
 3   브랜드구분코드             11725 non-null  object        
 4   MCT_SIGUNGU_NM      86278 non-null  object        
 5   업종                  86278 non-null  object        
 6   상권                  64931 non-null  object        
 7   개설일                 86278 non-null  datetime64[ns]
 8   폐업일                 2334 non-null   datetime64[ns]
 9   위도                  86278 non-null  float64       
 10  경도                  86278 non-null  float64       
 11  기준년월                86278 non-null  int64         
 12  운영개월수_구간            86278 non-null  object        
 13  월매출_구간              86278 non-null  object    

In [19]:
df_merged['기준년월_dt']

,기준년월_dt
0,2024-05
1,2023-04
2,2023-08
3,2024-02
4,2024-06
...,...
86273,2023-03
86274,2024-04
86275,2024-10
86276,2024-06


In [20]:
#영문 컬럼명 한글 변경
df_merged.rename(columns={'MCT_SIGUNGU_NM': '가맹점_지역'}, inplace=True)
#오기입 컬럼명 변경
df_merged.rename(columns={'배달매출비율': '취소율_구간', '배달매출비율.1': '배달매출비율','12개월_업종대비_평균고객수_비율':'12개월_업종대비_평균매출건수_비율','12개월_상권내_가맹점_비중':'12개월_동일업종_폐업가맹점_비중'}, inplace=True)

In [21]:
# 각 구간의 실제 구간 숫자만 보이게 변환 > % 상세부분 잘라내기
def convert_range_to_numeric(column):
    numeric_part = column.astype(str).str.split('_').str[0]
    numeric_part = numeric_part.fillna('0')
    return numeric_part.astype(int)

range_columns = ['운영개월수_구간', '월매출_구간', '월매출건수_구간', '월고객수_구간', '객단가_구간', '취소율_구간']

for col in range_columns:
    if col == '취소율_구간':
        df_merged[col] = df_merged[col].fillna('0')
    df_merged[col] = convert_range_to_numeric(df_merged[col])

In [22]:
df_merged.head()

,가맹점구분번호,가맹점주소,가맹점명,브랜드구분코드,가맹점_지역,업종,상권,개설일,폐업일,위도,경도,기준년월,운영개월수_구간,월매출_구간,월매출건수_구간,월고객수_구간,객단가_구간,취소율_구간,배달매출비율,12개월_업종대비_평균매출_비율,12개월_업종대비_평균매출건수_비율,12개월_업종내_매출_백분위,12개월_상권내_매출_백분위,12개월_동일업종_폐업가맹점_비중,12개월_동일상권_폐업가맹점_비중,남성20대이하_비중,남성30대_비중,남성40대_비중,남성50대_비중,남성60대이상_비중,여성20대이하_비중,여성30대_비중,여성40대_비중,여성50대_비중,여성60대이상_비중,재방문고객_비중,신규고객_비중,거주지고객_비중,직장이고객_비중,유동인구고객_비중,기준년월_dt
0,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,2013-03-20,NaT,37.569723,127.039314,202405,2,3,4,4,2,6,-999999.9,70.4,33.9,27.2,20.8,12.3,5.6,4.1480,9.044,11.0840,19.4480,24.2760,1.9520,4.256,5.2160,9.1520,11.4240,18.82,6.47,50.0,7.1,42.9,2024-05
1,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,2013-03-20,NaT,37.569723,127.039314,202304,2,4,4,4,2,6,-999999.9,74.6,22.9,11.5,21.3,11.3,6.0,3.9864,11.174,9.4224,16.9120,18.9052,2.6136,7.326,6.1776,11.0880,12.3948,16.51,5.05,25.0,6.3,68.8,2023-04
2,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,2013-03-20,NaT,37.569723,127.039314,202308,2,3,4,4,2,1,-999999.9,85.6,22.1,11.5,20.6,12.0,5.4,3.9060,11.214,10.6470,17.4510,19.7820,2.2940,6.586,6.2530,10.2490,11.6180,16.98,6.60,17.6,0.0,82.4,2023-08
3,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,2013-03-20,NaT,37.569723,127.039314,202402,2,3,4,4,1,1,-999999.9,90.5,35.8,27.8,22.2,12.3,5.0,4.7056,10.380,10.3800,21.8672,21.8672,2.0944,4.620,4.6200,9.7328,9.7328,16.95,9.60,15.8,5.3,78.9,2024-02
4,16184E93D9,서울 성동구 고산자로24길 16,성우**,NaN,서울 성동구,축산물,마장동,2013-03-20,NaT,37.569723,127.039314,202406,2,3,4,4,1,1,-999999.9,81.1,27.5,26.9,21.0,12.2,5.6,3.9345,7.869,10.8360,18.1245,23.6715,2.1655,4.331,5.9640,9.9755,13.0285,20.96,4.79,26.7,0.0,73.3,2024-06


In [23]:
df_merged['가맹점_지역'].unique()

array(['서울 성동구'], dtype=object)

In [24]:
df_merged['상권'].isnull().sum()

np.int64(21347)

In [25]:
df_merged['상권'].unique()

array(['마장동', nan, '한양대', '답십리', '왕십리', '뚝섬', '장한평자동차', '행당', '옥수', '신금호',
       '금남시장', '성수', '서면역', '건대입구', '동대문역사문화공원역', '압구정로데오', '오남', '화양시장',
       '방배역', '미아사거리', '풍산지구', '자양'], dtype=object)

In [26]:
df_merged['가맹점주소'].unique()

array(['서울 성동구 고산자로24길 16', '서울 성동구 마장로37길 53', '서울 성동구 마장로35다길 9', ...,
       '서울특별시 성동구 용답29길 28 ', '서울 성동구 독서당로 223', '서울 성동구 왕십리로 410'],
      dtype=object)

In [27]:
df_merged['가맹점주소'] = df_merged['가맹점주소'].str.replace('서울특별시', '서울')
df_merged['가맹점주소'] = df_merged['가맹점주소'].str.replace('  ', ' ')
df_merged['가맹점주소'].str[:6].unique()

array(['서울 성동구'], dtype=object)

In [28]:
df_merged['위도'].isnull().sum()

np.int64(0)

In [29]:
df_merged.to_csv("data_final.csv", index=False)

# 지역 전처리_추가 (상권 클러스터)

In [30]:
coords = df[["위도", "경도"]].dropna()  # NaN 제거
print(f"[check] 좌표 데이터: {coords.shape[0]}개 상권 사용")

inertia = []
K_range = range(5, 60, 5)  # 5~55까지 5단위

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=20)
    kmeans.fit(coords)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8,4))
plt.plot(K_range, inertia, marker="o", color="steelblue")
plt.xlabel("클러스터 개수 (k)")
plt.ylabel("Inertia")
plt.title("엘보우 방법 - 적정 k 탐색")
plt.grid(True)
plt.show()

silhouette_scores = []
K_range = range(5, 30, 2)  # 5~29까지 2단위

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=20)
    labels = kmeans.fit_predict(coords)
    score = silhouette_score(coords, labels)
    silhouette_scores.append(score)

plt.figure(figsize=(8,4))
plt.plot(K_range, silhouette_scores, marker="o", color="darkorange")
plt.title("실루엣 스코어 - 클러스터 품질 비교")
plt.xlabel("클러스터 수 (k)")
plt.ylabel("Silhouette Score")
plt.grid(True)
plt.show()

silhouette_scores = []
K_range = range(10, 21, 5)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=20)
    labels = kmeans.fit_predict(coords)
    score = silhouette_score(coords, labels)
    silhouette_scores.append(score)
    print(f"K={k}: 실루엣 점수 = {score:.3f}")

best_k = K_range[np.argmax(silhouette_scores)]
print(f"\n🎯 최적의 K(실루엣 기준): {best_k}")

kmeans = KMeans(n_clusters=15, random_state=42, n_init=20)
df["상권클러스터"] = kmeans.fit_predict(df[["위도", "경도"]])

print("\n[결과] 클러스터별 상권 수:")
print(df["상권클러스터"].value_counts().sort_index())

plt.figure(figsize=(8,6))
plt.scatter(df["경도"], df["위도"], c=df["상권클러스터"], cmap="tab20", s=20, alpha=0.7)
plt.title("상권 위치 기반 클러스터링 (k=15)")
plt.xlabel("경도")
plt.ylabel("위도")
plt.show()

df.to_csv('shinhan_final_1.csv', index=False, encoding="utf-8-sig")

[check] 좌표 데이터: 86278개 상권 사용


KeyboardInterrupt: 

# 머신러닝 전처리

## 원본과 출력 경로 기본 설정

In [31]:
from pathlib import Path
import pandas as pd
import numpy as np

# ==== 컬럼명 설정 ====
COL_STORE = '가맹점구분번호'
COL_DATE  = '기준년월'
COLS_RATIO = ['신규고객_비중_cleaned','재방문고객_비중_cleaned','유동인구고객_비중_cleaned']


## 데이터 불러오기 & 기본점검

In [43]:
print(f'df_ m.shape = {df_m.shape}')
df_m.head(3)

df_ m.shape = (86278, 10)


,가맹점구분번호,기준년월,신규고객_비중,재방문고객_비중,유동인구고객_비중,신규고객_비중_cleaned,재방문고객_비중_cleaned,유동인구고객_비중_cleaned,합계_3열,zero_sum_flag
0,000F03E44A,202301,100.0,0.0,100.0,100.0,0.0,100.0,200.0,0
1,000F03E44A,202302,0.0,0.0,-999999.9,0.0,0.0,0.0,0.0,1
2,000F03E44A,202303,0.0,0.0,-999999.9,0.0,0.0,0.0,0.0,1


## 공백 -> 0 치환, 숫자형으로 변환

In [33]:
df_m = df_m.copy()

# 공백을 NaN으로 통일 후 0 대체
for c in COLS_RATIO:
    if c not in df_m.columns:
        raise KeyError(f'필수 컬럼이 없습니다: {c}')
    # 문자열 공백 정리
    df_m[c] = df_m[c].replace({r'^\s*$': np.nan}, regex=True)
    # 숫자 변환
    df_m[c] = pd.to_numeric(df_m[c], errors='coerce')
    # NaN -> 0
    df_m[c] = df_m[c].fillna(0)

df_m[COLS_RATIO].head(5)


,신규고객_비중_cleaned,재방문고객_비중_cleaned,유동인구고객_비중_cleaned
0,100.00,0.0,100.0
1,0.00,0.0,0.0
2,0.00,0.0,0.0
3,50.00,0.0,0.0
4,33.33,0.0,100.0


## 월별 합산 및 0합 플래그 생성
- 행단위로 합계열 = 신규 + 재방문 + 유동인구 계산,
- 합계가 0이면 플래그 생성

In [34]:
df_m['합계_3열'] = df_m[COLS_RATIO].sum(axis=1, skipna=True)
df_m['zero_sum_flag'] = (df_m['합계_3열'] == 0).astype(int)

df_m[[COL_STORE, COL_DATE, '합계_3열', 'zero_sum_flag'] + COLS_RATIO].head(8)


,가맹점구분번호,기준년월,합계_3열,zero_sum_flag,신규고객_비중_cleaned,재방문고객_비중_cleaned,유동인구고객_비중_cleaned
0,000F03E44A,202301,200.00,0,100.00,0.0,100.0
1,000F03E44A,202302,0.00,1,0.00,0.0,0.0
2,000F03E44A,202303,0.00,1,0.00,0.0,0.0
3,000F03E44A,202304,50.00,0,50.00,0.0,0.0
4,000F03E44A,202305,133.33,0,33.33,0.0,100.0
5,000F03E44A,202306,125.00,0,25.00,0.0,100.0
6,000F03E44A,202307,0.00,1,0.00,0.0,0.0
7,000F03E44A,202308,0.00,1,0.00,0.0,0.0


## 가맹점 단위 삭제
- 0 플래그를 가맹점별로 합산
- 0 플래그 합산이 3이상인 경우 전체 행 삭제

In [35]:
flag_counts = df_m.groupby(COL_STORE)['zero_sum_flag'].sum().rename('zero_sum_cnt')
to_drop_ids = flag_counts[flag_counts >= 3].index

summary = (
    flag_counts.to_frame()
    .assign(삭제대상=lambda s: s['zero_sum_cnt'] >= 3)
    .sort_values('zero_sum_cnt', ascending=False)
)

print(f'전체 가맹점 수: {flag_counts.shape[0]}')
print(f'삭제 대상 가맹점 수(>=3회): {len(to_drop_ids)}')
summary.head(10)


전체 가맹점 수: 4172
삭제 대상 가맹점 수(>=3회): 259


,zero_sum_cnt,삭제대상
가맹점구분번호,,
62A9B7888B,23,True
5B537AFFD8,23,True
E0B79043DA,23,True
EED0404EC2,23,True
261CA88AD3,23,True
81E3DAF03B,23,True
A6964F8879,23,True
708697C149,23,True
F4D804EFA3,23,True


## 데이터 필터링 및 저장

In [36]:
df_clean = df_m[~df_m[COL_STORE].isin(to_drop_ids)].copy()
print('정제 전/후 행 수:', len(df), '->', len(df_clean))
df_clean.to_csv( '가맹점_고객비중_정제.csv' , index=False, encoding='utf-8-sig')


정제 전/후 행 수: 86278 -> 80705


## 점검 : 삭제 기준이 잘 적용되었는지 확인

In [37]:
post_flag = (
    df_clean.groupby(COL_STORE)['zero_sum_flag'].sum()
)
violations = post_flag[post_flag >= 3]
print('잔여 위반 가맹점 수:', violations.shape[0])
violations.head()


잔여 위반 가맹점 수: 0


,zero_sum_flag
가맹점구분번호,


In [41]:
df_clean

,가맹점구분번호,기준년월_dt,신규고객_비중_cleaned,재방문고객_비중_cleaned,유동인구고객_비중_cleaned
0,002816BA73,2023-01-01,7.20,12.85,66.7
1,002816BA73,2023-02-01,7.14,12.07,58.6
2,002816BA73,2023-03-01,9.50,12.35,72.7
3,002816BA73,2023-04-01,7.87,12.73,66.7
4,002816BA73,2023-05-01,7.23,12.53,75.8
...,...,...,...,...,...
80700,FFF0DAC445,2024-08-01,6.24,9.60,63.2
80701,FFF0DAC445,2024-09-01,8.07,9.62,62.3
80702,FFF0DAC445,2024-10-01,7.48,10.27,67.2
80703,FFF0DAC445,2024-11-01,7.96,10.28,62.6


## 시계열 예측 준비
- 기준년월을 datetime 형태로 변환
- 가맹점 단위 패널 형태 유지
- 개별학습 / 하나의 전역 모델 선택

In [42]:
# 기준년월이 202301, 202302 같은 형태로 되어 있을 때
df_clean['기준년월_dt'] = (
    df_clean['기준년월']
    .astype(str)                              # 문자열로 변환
    .str.zfill(6)                             # 혹시 6자 미만일 경우 보완
    .apply(lambda x: f"{x[:4]}-{x[4:6]}-01")  # 'YYYY-MM-01' 형태로 변환
)
df_clean['기준년월_dt'] = pd.to_datetime(df_clean['기준년월_dt'], format='%Y-%m-%d', errors='coerce')

# 시계열 정렬
df_clean = df_clean.sort_values(['가맹점구분번호', '기준년월_dt']).reset_index(drop=True)

df_clean = df_clean[['가맹점구분번호','기준년월_dt','신규고객_비중_cleaned','재방문고객_비중_cleaned','유동인구고객_비중_cleaned']]
df_clean.to_csv('가맹점_고객비중_정제.csv',index=False)

KeyError: '기준년월'

## 시계열 학습 데이터 분할 예시
- 21개월 / 3개월



- 머신러닝 정확도 비교를 위해 값을 수정해가며 22/2, 23/1 등으로 해보기

In [40]:
# 예측(홀드아웃) 대상 월 3개
TEST_MONTHS = pd.to_datetime(["2024-10-01", "2024-11-01", "2024-12-01"])

# 정렬
df_sorted = df_clean.sort_values(["가맹점구분번호", "기준년월_dt"]).reset_index(drop=True)

# 분할
train_df = df_sorted[~df_sorted["기준년월_dt"].isin(TEST_MONTHS)].copy()  # 2023-01 ~ 2024-09 (21개월)
test_df  = df_sorted[df_sorted["기준년월_dt"].isin(TEST_MONTHS)].copy()   # 2024-10 ~ 2024-12 (3개월)


# === 21개월 학습 + 3개월 예측 분할 후, 불완전한 가맹점 제거 ===

# 가맹점별 개수 확인
counts = df_sorted.groupby("가맹점구분번호")["기준년월_dt"].nunique().rename("month_count")

# 24개월(2023-01~2024-12)이 모두 존재하지 않는 가맹점 제거
valid_store_ids = counts[counts == 24].index

# 필터링
df_complete = df_sorted[df_sorted["가맹점구분번호"].isin(valid_store_ids)].copy()

print(f"전체 가맹점 수: {counts.shape[0]}")
print(f"24개월 모두 존재하는 가맹점 수: {len(valid_store_ids)}")
print(f"삭제된 가맹점 수: {counts.shape[0] - len(valid_store_ids)}")

# 학습/예측 분할 다시 적용
train_df = df_complete[~df_complete["기준년월_dt"].isin(TEST_MONTHS)].copy()
test_df  = df_complete[df_complete["기준년월_dt"].isin(TEST_MONTHS)].copy()

chk_train = train_df.groupby("가맹점구분번호")["기준년월_dt"].nunique().rename("train_months")
chk_test  = test_df.groupby("가맹점구분번호")["기준년월_dt"].nunique().rename("test_months")
check = pd.concat([chk_train, chk_test], axis=1).fillna(0).astype(int)


# (선택) 모든 가맹점이 21/3을 만족하는지 간단 확인
assert (check["train_months"] == 21).all(), "일부 가맹점의 학습 월 수가 21이 아닙니다."
assert (check["test_months"]  == 3 ).all(), "일부 가맹점의 예측 월 수가 3이 아닙니다."

print(train_df.shape, test_df.shape)
train_df.head(3), test_df.head(3)

print(f"최종 학습 데이터: {train_df.shape}, 예측 데이터: {test_df.shape}")




전체 가맹점 수: 3913
24개월 모두 존재하는 가맹점 수: 2838
삭제된 가맹점 수: 1075
(59598, 5) (8514, 5)
최종 학습 데이터: (59598, 5), 예측 데이터: (8514, 5)
